# Description of the Project and Dataset

## Project Description
The Global Superstore Dashboard project uses Plotly's Dash framework to create an interactive web application that visualizes various aspects of global sales data. The primary goal is to showcase interactive visualization skills using Plotly and Dash. The app includes the following pages:

1. Overview: A histogram displaying sales over time.
2. Sales Analysis: A dropdown menu to select a category and view sales by sub-category.
4. Customer Insights: A search box to filter sales data by customer name and view sales by customer.

## Dataset Description
The dataset used in this project is the Global Superstore dataset, which contains information about orders, products, customers, and shipping details. The dataset includes the following columns:

- **Order ID:** Unique order identifier
- **Order Date:** Date the order was placed
- **Ship Date:** Date the order was shipped
- **Ship Mode:** Mode of shipping
- **Customer ID:** Unique customer identifier
- **Customer Name:** Name of the customer
- **Segment:** Segment to which the customer belongs
- **Country:** Country of the customer
- **City**: City of the customer
- **State**: State of the customer
- **Postal Code:** Postal code of the customer
- **Region:** Region of the customer
- **Product ID:** Unique product identifier
- **Category** Category of the product
- **Sub-Category:** Sub-category of the product
- **Product Name:** Name of the product
- **Sales:** Sales amount
- **Quantity:** Quantity of the product ordered
- **Discount:** Discount applied on the product
- **Profit:** Profit earned from the product

This project demonstrates the use of Plotly and Dash to create an interactive data visualization web application, providing valuable insights into global sales data.

## Step 1: Load and Prepare the Data
First, let's load and prepare the dataset:

In [10]:
import pandas as pd
import os

# Load the dataset
path = r'C:\Users\hassa\OneDrive\المستندات\Machine learning files\Global_Superstore\Global Superstore.xls'
data = pd.read_excel(path)

# Display the first few rows of the dataset
data.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       51290 non-null  object        
 1   Order Date     51290 non-null  datetime64[ns]
 2   Ship Date      51290 non-null  datetime64[ns]
 3   Ship Mode      51290 non-null  object        
 4   Customer ID    51290 non-null  object        
 5   Customer Name  51290 non-null  object        
 6   Segment        51290 non-null  object        
 7   City           51290 non-null  object        
 8   State          51290 non-null  object        
 9   Country        51290 non-null  object        
 10  Postal Code    9994 non-null   float64       
 11  Market         51290 non-null  object        
 12  Region         51290 non-null  object        
 13  Product ID     51290 non-null  object        
 14  Category       51290 non-null  object        
 15  Sub-Category   5129

## Step 2: Create a Dash App
We'll create a Dash app with multiple pages, each showcasing different aspects of the data. The app will include filters, search boxes, and other interactive components to provide a comprehensive view of the data.

In [16]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd


# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

# Define the layout of the app
app.layout = dbc.Container([
    dbc.NavbarSimple(
        brand="Global Superstore Dashboard",
        brand_href="/",
        color="primary",
        dark=True,
        children=[
            dbc.NavItem(dbc.NavLink("Overview", href="/overview")),
            dbc.NavItem(dbc.NavLink("Sales Analysis", href="/sales-analysis")),
            dbc.NavItem(dbc.NavLink("Customer Insights", href="/customer-insights")),
        ],
    ),
    dcc.Location(id='url', refresh=False),
    dbc.Container(id='page-content', className='mt-4')
])

# Callback to update the page content based on the URL
@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == '/sales-analysis':
        return sales_analysis_layout()
    elif pathname == '/customer-insights':
        return customer_insights_layout()
    else:
        return overview_layout()

# Define the overview layout
def overview_layout():
    return html.Div([
        html.H2("Overview"),
        dcc.Graph(
            id='overview-sales',
            figure=px.histogram(data, x='Order Date', y='Sales', title='Sales Over Time')
        )
    ])

# Define the sales analysis layout
def sales_analysis_layout():
    return html.Div([
        html.H2("Sales Analysis"),
        html.Label("Select Category:"),
        dcc.Dropdown(
            id='category-dropdown',
            options=[{'label': category, 'value': category} for category in data['Category'].unique()],
            value=data['Category'].unique()[0]
        ),
        html.Label("Select Sub-Category:"),
        dcc.Dropdown(
            id='sub-category-dropdown'
        ),
        dcc.Graph(id='sales-category-graph')
    ])

# Callback to update the sub-category dropdown based on the selected category
@app.callback(
    Output('sub-category-dropdown', 'options'),
    [Input('category-dropdown', 'value')]
)
def set_sub_category_options(selected_category):
    sub_categories = data[data['Category'] == selected_category]['Sub-Category'].unique()
    return [{'label': sub_category, 'value': sub_category} for sub_category in sub_categories]

# Callback to update the sub-category dropdown value based on the selected category
@app.callback(
    Output('sub-category-dropdown', 'value'),
    [Input('sub-category-dropdown', 'options')]
)
def set_sub_category_value(available_options):
    return available_options[0]['value']

# Callback to update the sales analysis graph based on the selected sub-category
@app.callback(
    Output('sales-category-graph', 'figure'),
    [Input('sub-category-dropdown', 'value')]
)
def update_sales_category_graph(selected_sub_category):
    filtered_data = data[data['Sub-Category'] == selected_sub_category]
    figure = px.bar(filtered_data, x='Order Date', y='Sales', title=f'Sales for {selected_sub_category}')
    return figure

# Define the customer insights layout
def customer_insights_layout():
    return html.Div([
        html.H2("Customer Insights"),
        html.Label("Search Customer:"),
        dcc.Input(id='customer-search', type='text', placeholder='Enter customer name'),
        dcc.Graph(id='customer-sales-graph')
    ])

# Callback to update the customer sales graph based on the search input
@app.callback(
    Output('customer-sales-graph', 'figure'),
    [Input('customer-search', 'value')]
)
def update_customer_sales_graph(search_value):
    if search_value:
        filtered_data = data[data['Customer Name'].str.contains(search_value, case=False)]
    else:
        filtered_data = data
    figure = px.scatter(filtered_data, x='Order Date', y='Sales', color='Customer Name', title='Sales by Customer')
    return figure

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
